In [137]:
import pandas as pd
import numpy as np

In [138]:
pd.set_option('display.max_columns', None)

In [139]:
df = pd.read_csv("B2B_updated_final3.csv")
df2 = pd.read_csv("game_data1.csv", header=1)

In [142]:
#Defining column names for game data
df2 = df2.rename(columns = {
    'Unnamed: 0':'game_id', 
    'GP':'game_played', 
    'Date':'date', 
    'Unnamed: 2_level_1':'home_away', 
    'Opponent':'opponent', 
    'GF':'team_GF', 
    'GA':'team_GA', 
    'Unnamed: 6_level_1':'win_loss',
    'Unnamed: 7_level_1':'extra_time', 
    'Unnamed: 8_level_1':'extra_info', 
    'S':'team_shots', 
    'PIM':'team_PIM', 
    'PPG':'team_PPG', 
    'PPO':'team_PPO', 
    'SHG':'team_SHG', 
    'Unnamed: 14_level_1':'extra_info_1', 
    'S.1':'opp_shots', 
    'PIM.1':'opp_PIM', 
    'PPG.1':'opp_PPG', 
    'PPO.1':'opp_PPO', 
    'SHG.1':'opp_SHG', 
    'Unnamed: 20_level_1':'extra_info_2', 
    'CF':'team_CF', 
    'CA':'team_CA', 
    'CF%':'team_CF_percentage', 
    'FF':'team_FF', 
    'FA':'team_FA', 
    'FF%':'team_FF_percentage', 
    'FOW':'team_FOW', 
    'FOL':'team_FOL', 
    'FO%':'team_FO_win_percentage', 
    'oZS%':'team_oZS_percentage', 
    'PDO':'team_PDO', 
    'Unnamed: 33':'team', 
    'Unnamed: 34':'season'
})

team_names = {
    'VEG': 'Vegas Golden Knights',
    'LAK': 'Los Angeles Kings',
    'TBL': 'Tampa Bay Lightning',
    'NYR': 'New York Rangers',
    'SEA': 'Seattle Kraken',
    'ANA': 'Anaheim Ducks',
    'SJS': 'San Jose Sharks',
    'CBJ': 'Columbus Blue Jackets',
    'CAR': 'Carolina Hurricanes',
    'CHI': 'Chicago Blackhawks',
    'COL': 'Colorado Avalanche',
    'VAN': 'Vancouver Canucks',
    'EDM': 'Edmonton Oilers',
    'TOR': 'Toronto Maple Leafs',
    'MTL': 'Montreal Canadiens',
    'BOS': 'Boston Bruins',
    'WSH': 'Washington Capitals',
    'OTT': 'Ottawa Senators',
    'BUF': 'Buffalo Sabres',
    'CGY': 'Calgary Flames',
    'MIN': 'Minnesota Wild',
    'FLA': 'Florida Panthers',
    'NYI': 'New York Islanders',
    'DAL': 'Dallas Stars',
    'NSH': 'Nashville Predators',
    'DET': 'Detroit Red Wings',
    'ARI': 'Arizona Coyotes',
    'PIT': 'Pittsburgh Penguins',
    'NJD': 'New Jersey Devils',
    'PHI': 'Philadelphia Flyers',
    'STL': 'St. Louis Blues',
    'WPG': 'Winnipeg Jets',
    'ATL': 'Atlanta Thrashers',  # Adding Atlanta Thrashers (now Winnipeg Jets)
    'PHX': 'Phoenix Coyotes',    # Adding Phoenix Coyotes (now Arizona Coyotes)
}

#Mapping team names to game data based on abbrivation
df2['team'] = df2['team'].map(team_names)

In [143]:
#cleaning the data
df2['season'] = df2['season'].apply(lambda x: str(int(x) - 1) + str(x))
df2 = df2[df2['game_played'] != 'GP']
df2['home_away'] = np.where(df2['home_away'] == '@', 'away', 'home')

In [144]:
#The date columns in the scraped dataframes were not consistent
df['date'] = pd.to_datetime(df['date'])

dc3 = pd.to_datetime(df2['date'], format = '%Y-%m-%d', errors = 'coerce')
dc4 = pd.to_datetime(df2['date'], format = '%d/%m/%y', errors = 'coerce')
df2['date'] = dc3.fillna(dc4)

In [145]:
#matching string values from both dataframes
from fuzzywuzzy import process

# Choose one dataframe as the "standard"
unique_teams = df2['team'].unique()

def get_closest_match(x):
    closest_match, score = process.extractOne(x, unique_teams)
    # Only return the match if it's above a certain confidence threshold, e.g., 90
    if score > 90:
        return closest_match
    return x

df['homename'] = df['homename'].apply(get_closest_match)
df['awayname'] = df['awayname'].apply(get_closest_match)

In [146]:
print(set(df['homename'].unique()) - set(df2['team'].unique()))
print(set(df2['team'].unique()) - set(df['homename'].unique()))

set()
set()


In [153]:
print(df2[df2.duplicated()])

Empty DataFrame
Columns: [game_id, game_played, date, home_away, opponent, team_GF, team_GA, win_loss, extra_time, extra_info, team_shots, team_PIM, team_PPG, team_PPO, team_SHG, extra_info_1, opp_shots, opp_PIM, opp_PPG, opp_PPO, opp_SHG, extra_info_2, team_CF, team_CA, team_CF_percentage, team_FF, team_FA, team_FF_percentage, team_FOW, team_FOL, team_FO_win_percentage, team_oZS_percentage, team_PDO, team, season, back_to_back, 3_in_4, 5_in_8]
Index: []


### No duplicate fixtures or mismatched team names

In [147]:

df2.sort_values(['team', 'season', 'date'], inplace=True)
#calculate b2b games
df2['date_diff_2'] = df2.groupby(['team', 'season'])['date'].diff().dt.days
df2['b2b'] = (df2['date_diff_2'] == 1).astype(int)

# Calculate for 3 games in 4 days
df2['date_2_before'] = df2.groupby(['team', 'season'])['date'].shift(2)
df2['3_in_4'] = ((df2['date'] - df2['date_2_before']).dt.days <= 3).astype(int)

# Calculate for 5 games in 8 days
df2['date_4_before'] = df2.groupby(['team', 'season'])['date'].shift(4)
df2['5_in_8'] = ((df2['date'] - df2['date_4_before']).dt.days <= 7).astype(int)

# Drop temporary columns
df2.drop(['date_diff_2', 'date_2_before', 'date_4_before'], axis=1, inplace=True)


In [9]:
df2.head(5)

,game_id,game_played,date,home_away,opponent,team_GF,team_GA,win_loss,extra_time,extra_info,team_shots,team_PIM,team_PPG,team_PPO,team_SHG,extra_info_1,opp_shots,opp_PIM,opp_PPG,opp_PPO,opp_SHG,extra_info_2,team_CF,team_CA,team_CF_percentage,team_FF,team_FA,team_FF_percentage,team_FOW,team_FOL,team_FO_win_percentage,team_oZS_percentage,team_PDO,team,season,is_b2b_game
0,0,1,2009-10-03,home,San Jose Sharks,1,4,L,NaN,NaN,25,34,1,NaN,0,NaN,37,46,1,NaN,1,NaN,29,43,40.3,23,37,38.3,14,13,51.9,47.7,93.1,Anaheim Ducks,20092010,0
1,1,2,2009-10-06,away,Minnesota Wild,3,4,L,OT,NaN,19,19,1,NaN,0,NaN,32,13,3,NaN,0,NaN,29,47,38.2,23,33,41.1,18,33,35.3,37.5,107.5,Anaheim Ducks,20092010,0
2,2,3,2009-10-08,away,Boston Bruins,6,1,W,NaN,NaN,30,12,2,NaN,1,NaN,34,4,0,NaN,0,NaN,38,49,43.7,30,39,43.5,26,14,65.0,40.0,110.3,Anaheim Ducks,20092010,0
3,3,4,2009-10-10,away,Philadelphia Flyers,3,2,W,SO,NaN,33,16,0,NaN,0,NaN,35,10,2,NaN,0,NaN,46,48,48.9,33,34,49.3,21,17,55.3,53.5,107.7,Anaheim Ducks,20092010,0
4,4,5,2009-10-11,away,New York Rangers,0,3,L,NaN,NaN,18,10,0,NaN,0,NaN,38,8,2,NaN,1,NaN,34,56,37.8,23,45,33.8,20,20,50.0,37.9,100.0,Anaheim Ducks,20092010,1


In [129]:
print(df2.isnull().sum())

game_id                       0
game_played                   0
date                          0
home_away                     0
opponent                      0
team_GF                       0
team_GA                       0
win_loss                      0
extra_time                28904
extra_info                37976
team_shots                    0
team_PIM                      0
team_PPG                      0
team_PPO                  16118
team_SHG                      0
extra_info_1              37976
opp_shots                     0
opp_PIM                       0
opp_PPG                       0
opp_PPO                   16118
opp_SHG                       0
extra_info_2              37976
team_CF                      32
team_CA                      32
team_CF_percentage           32
team_FF                      32
team_FA                       0
team_FF_percentage           32
team_FOW                      0
team_FOL                      0
team_FO_win_percentage       32
team_oZS

In [130]:
filtered_df = df2[df2['team_CF'].isna() | 
                  df2['team_CA'].isna() |
                  df2['team_CF_percentage'].isna() | 
                  df2['team_FF'].isna() | 
                  df2['team_FF_percentage'].isna() |
                  df2['team_FO_win_percentage'].isna()]
                

In [131]:
filtered_df.head(5)

,game_id,game_played,date,home_away,opponent,team_GF,team_GA,win_loss,extra_time,extra_info,team_shots,team_PIM,team_PPG,team_PPO,team_SHG,extra_info_1,opp_shots,opp_PIM,opp_PPG,opp_PPO,opp_SHG,extra_info_2,team_CF,team_CA,team_CF_percentage,team_FF,team_FA,team_FF_percentage,team_FOW,team_FOL,team_FO_win_percentage,team_oZS_percentage,team_PDO,team,season,is_b2b_game
181,189,18,2011-11-16,away,Los Angeles Kings,1,2,L,SO,NaN,24,12,1,NaN,0,NaN,37,8,1,NaN,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,Anaheim Ducks,20112012,0
2240,2346,54,2010-02-02,home,Washington Capitals,1,4,L,NaN,NaN,42,8,1,NaN,0,NaN,26,12,0,NaN,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,Boston Bruins,20092010,0
3317,3474,55,2010-02-03,home,Ottawa Senators,2,4,L,NaN,NaN,36,10,0,NaN,0,NaN,38,10,0,NaN,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,Buffalo Sabres,20092010,0
4343,4549,6,2009-10-14,home,Pittsburgh Penguins,2,3,L,SO,NaN,29,8,0,NaN,0,NaN,38,6,0,NaN,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,Carolina Hurricanes,20092010,0
4427,4637,8,2010-10-27,home,Washington Capitals,0,3,L,NaN,NaN,29,6,0,NaN,0,NaN,33,10,0,NaN,1,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,Carolina Hurricanes,20102011,0


#### Checked a few games to understand why there are 18 nan values for advanced stats and those games have no advanced stats available for that game. (Most of the games in 2009-2011 era)


#### Rest all the data is consistent and all of the missing values are due to no info available on hoceky reference. 